In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
1559207,2021-08-14,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1559208,2021-08-15,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1559209,2021-08-16,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1559210,2021-08-17,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
40923,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
40925,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
40927,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
40929,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
40931,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
952491,2021-08-14,Arkansas,Arkansas,5001,2822,47.00,5000,Arkansas,AR,Arkansas,State,3017804
952493,2021-08-15,Arkansas,Arkansas,5001,2842,48.00,5000,Arkansas,AR,Arkansas,State,3017804
952495,2021-08-16,Arkansas,Arkansas,5001,2845,48.00,5000,Arkansas,AR,Arkansas,State,3017804
952497,2021-08-17,Arkansas,Arkansas,5001,2870,48.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
1559207,2021-08-14,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1559208,2021-08-15,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1559209,2021-08-16,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1559210,2021-08-17,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-08-18') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
575,2021-08-18,Snohomish,Washington,46936,632.00,53061,WA,County,822083,5709.40,76.88
1148,2021-08-18,Cook,Illinois,578907,11211.00,17031,IL,County,5150233,11240.40,217.68
1720,2021-08-18,Orange,California,297163,5165.00,6059,CA,County,3175692,9357.43,162.64
2291,2021-08-18,Maricopa,Arizona,613962,10612.00,4013,AZ,County,4485414,13687.97,236.59
2862,2021-08-18,Los Angeles,California,1360366,24967.00,6037,CA,County,10039107,13550.67,248.70
...,...,...,...,...,...,...,...,...,...,...,...
1558095,2021-08-18,Wheeler,Oregon,56,1.00,41069,OR,County,1332,4204.20,75.08
1558405,2021-08-18,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00
1558684,2021-08-18,Esmeralda,Nevada,41,0.00,32009,NV,County,873,4696.45,0.00
1558959,2021-08-18,Loving,Texas,5,0.00,48301,TX,County,169,2958.58,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
575,2021-08-18,Snohomish,Washington,46936,632.00,53061,WA,County,822083,5709.40,76.88,76.88,5709.40
1148,2021-08-18,Cook,Illinois,578907,11211.00,17031,IL,County,5150233,11240.40,217.68,217.68,11240.40
1720,2021-08-18,Orange,California,297163,5165.00,6059,CA,County,3175692,9357.43,162.64,162.64,9357.43
2291,2021-08-18,Maricopa,Arizona,613962,10612.00,4013,AZ,County,4485414,13687.97,236.59,236.59,13687.97
2862,2021-08-18,Los Angeles,California,1360366,24967.00,6037,CA,County,10039107,13550.67,248.70,248.70,13550.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1558095,2021-08-18,Wheeler,Oregon,56,1.00,41069,OR,County,1332,4204.20,75.08,75.08,4204.20
1558405,2021-08-18,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00,0.00,4044.12
1558684,2021-08-18,Esmeralda,Nevada,41,0.00,32009,NV,County,873,4696.45,0.00,0.00,4696.45
1558959,2021-08-18,Loving,Texas,5,0.00,48301,TX,County,169,2958.58,0.00,0.00,2958.58


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
996184,2021-08-18,Hale,Alabama,2483,79.00,1065,AL,County,14651,16947.65,539.21,539.21,16947.65,1
1084829,2021-08-18,Clarke,Alabama,3967,63.00,1025,AL,County,23622,16793.67,266.70,266.70,16793.67,2
690011,2021-08-18,Etowah,Alabama,16119,377.00,1055,AL,County,102268,15761.53,368.64,368.64,15761.53,3
578616,2021-08-18,Franklin,Alabama,4935,83.00,1059,AL,County,31362,15735.60,264.65,264.65,15735.60,4
778581,2021-08-18,Lowndes,Alabama,1519,55.00,1085,AL,County,9726,15617.93,565.49,565.49,15617.93,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
950466,2021-08-18,Converse,Wyoming,1279,20.00,56009,WY,County,13822,9253.36,144.70,144.70,9253.36,19
950975,2021-08-18,Sublette,Wyoming,890,10.00,56035,WY,County,9831,9053.00,101.72,101.72,9053.00,20
1183111,2021-08-18,Lincoln,Wyoming,1640,14.00,56023,WY,County,19830,8270.30,70.60,70.60,8270.30,21
1317927,2021-08-18,Niobrara,Wyoming,183,3.00,56027,WY,County,2356,7767.40,127.33,127.33,7767.40,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
778581,2021-08-18,Lowndes,Alabama,1519,55.00,1085,AL,County,9726,15617.93,565.49,565.49,15617.93,5,1
996184,2021-08-18,Hale,Alabama,2483,79.00,1065,AL,County,14651,16947.65,539.21,539.21,16947.65,1,2
839898,2021-08-18,Greene,Alabama,1039,39.00,1063,AL,County,8111,12809.76,480.83,480.83,12809.76,42,3
330140,2021-08-18,Walker,Alabama,8488,291.00,1127,AL,County,63521,13362.51,458.12,458.12,13362.51,32,4
687963,2021-08-18,Clay,Alabama,1783,60.00,1027,AL,County,13235,13471.85,453.34,453.34,13471.85,28,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1183111,2021-08-18,Lincoln,Wyoming,1640,14.00,56023,WY,County,19830,8270.30,70.60,70.60,8270.30,21,19
1153434,2021-08-18,Uinta,Wyoming,2686,14.00,56041,WY,County,20226,13279.94,69.22,69.22,13279.94,3,20
837857,2021-08-18,Hot Springs,Wyoming,412,3.00,56017,WY,County,4413,9336.05,67.98,67.98,9336.05,18,21
775515,2021-08-18,Albany,Wyoming,4990,19.00,56001,WY,County,38880,12834.36,48.87,48.87,12834.36,5,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,76.88,5709.40,21,24
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,76.88,5709.40,21,24
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,76.88,5709.40,21,24
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,76.88,5709.40,21,24
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,76.88,5709.40,21,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1555581,2021-08-14,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5
1555582,2021-08-15,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5
1555583,2021-08-16,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5
1555584,2021-08-17,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day
992052,2020-03-30,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,539.21,16947.65,2,1,1.00
992053,2020-03-31,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,539.21,16947.65,2,1,0.00
992054,2020-04-01,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,539.21,16947.65,2,1,0.00
992055,2020-04-02,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,539.21,16947.65,2,1,0.00
992056,2020-04-03,Hale,Alabama,3,0.00,1065,AL,County,14651,20.48,0.00,539.21,16947.65,2,1,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1203075,2021-08-14,Crook,Wyoming,489,13.00,56011,WY,County,7584,6447.78,171.41,171.41,6606.01,8,23,0.00
1203076,2021-08-15,Crook,Wyoming,489,13.00,56011,WY,County,7584,6447.78,171.41,171.41,6606.01,8,23,0.00
1203077,2021-08-16,Crook,Wyoming,493,13.00,56011,WY,County,7584,6500.53,171.41,171.41,6606.01,8,23,4.00
1203078,2021-08-17,Crook,Wyoming,498,13.00,56011,WY,County,7584,6566.46,171.41,171.41,6606.01,8,23,5.00


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
775463,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,565.49,15617.93,1,5,1.00,0.00
775464,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,565.49,15617.93,1,5,0.00,0.00
775465,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,565.49,15617.93,1,5,0.00,0.00
775466,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,565.49,15617.93,1,5,0.00,0.00
775467,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,565.49,15617.93,1,5,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326486,2021-08-14,Teton,Wyoming,4068,11.00,56039,WY,County,23464,17337.20,46.88,46.88,17601.43,23,1,0.00,0.00
326487,2021-08-15,Teton,Wyoming,4068,11.00,56039,WY,County,23464,17337.20,46.88,46.88,17601.43,23,1,0.00,0.00
326488,2021-08-16,Teton,Wyoming,4107,11.00,56039,WY,County,23464,17503.41,46.88,46.88,17601.43,23,1,39.00,0.00
326489,2021-08-17,Teton,Wyoming,4116,11.00,56039,WY,County,23464,17541.77,46.88,46.88,17601.43,23,1,9.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-08-18') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
394340,2021-08-18,Imperial,California,32686,745.00,6025,CA,County,181215,18037.14,411.11,411.11,18037.14,1,2,31.00,0.00
2862,2021-08-18,Los Angeles,California,1360366,24967.00,6037,CA,County,10039107,13550.67,248.70,248.70,13550.67,2,5,3974.00,32.00
195922,2021-08-18,San Bernardino,California,324147,5302.00,6071,CA,County,2180085,14868.55,243.20,243.20,14868.55,3,4,785.00,1.00
692011,2021-08-18,Inyo,California,1509,39.00,6027,CA,County,18039,8365.21,216.20,216.20,8365.21,4,31,0.00,0.00
87401,2021-08-18,Stanislaus,California,70193,1112.00,6099,CA,County,550660,12747.07,201.94,201.94,12747.07,5,9,283.00,3.00
69485,2021-08-18,San Joaquin,California,85520,1489.00,6077,CA,County,762148,11220.92,195.37,195.37,11220.92,6,16,213.00,1.00
40392,2021-08-18,Riverside,California,331619,4689.00,6065,CA,County,2470546,13422.90,189.80,189.80,13422.90,7,7,1277.00,3.00
108438,2021-08-18,Tulare,California,62710,856.00,6107,CA,County,466195,13451.45,183.61,183.61,13451.45,8,6,200.00,0.00
626877,2021-08-18,Merced,California,34572,492.00,6047,CA,County,277680,12450.30,177.18,177.18,12450.30,9,10,14.00,0.00
39332,2021-08-18,Fresno,California,119941,1755.00,6019,CA,County,999101,12004.89,175.66,175.66,12004.89,10,12,640.00,2.00


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
1472425,2021-08-18,Lassen,California,6080,26.00,6035,CA,County,30573,19886.83,85.04,85.04,19886.83,35,1,0.00,0.00
394340,2021-08-18,Imperial,California,32686,745.00,6025,CA,County,181215,18037.14,411.11,411.11,18037.14,1,2,31.00,0.00
839840,2021-08-18,Kings,California,26253,258.00,6031,CA,County,152940,17165.56,168.69,168.69,17165.56,11,3,113.00,2.00
195922,2021-08-18,San Bernardino,California,324147,5302.00,6071,CA,County,2180085,14868.55,243.20,243.20,14868.55,3,4,785.00,1.00
2862,2021-08-18,Los Angeles,California,1360366,24967.00,6037,CA,County,10039107,13550.67,248.70,248.70,13550.67,2,5,3974.00,32.00
108438,2021-08-18,Tulare,California,62710,856.00,6107,CA,County,466195,13451.45,183.61,183.61,13451.45,8,6,200.00,0.00
40392,2021-08-18,Riverside,California,331619,4689.00,6065,CA,County,2470546,13422.90,189.80,189.80,13422.90,7,7,1277.00,3.00
246501,2021-08-18,Kern,California,118794,1441.00,6029,CA,County,900202,13196.37,160.08,160.08,13196.37,13,8,434.00,5.00
87401,2021-08-18,Stanislaus,California,70193,1112.00,6099,CA,County,550660,12747.07,201.94,201.94,12747.07,5,9,283.00,3.00
626877,2021-08-18,Merced,California,34572,492.00,6047,CA,County,277680,12450.30,177.18,177.18,12450.30,9,10,14.00,0.00


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
394340,2021-08-18,Imperial,California,32686,745.00,6025,CA,County,181215,18037.14,411.11,411.11,18037.14,1,2,31.00,0.00
2862,2021-08-18,Los Angeles,California,1360366,24967.00,6037,CA,County,10039107,13550.67,248.70,248.70,13550.67,2,5,3974.00,32.00
195922,2021-08-18,San Bernardino,California,324147,5302.00,6071,CA,County,2180085,14868.55,243.20,243.20,14868.55,3,4,785.00,1.00
692011,2021-08-18,Inyo,California,1509,39.00,6027,CA,County,18039,8365.21,216.20,216.20,8365.21,4,31,0.00,0.00
87401,2021-08-18,Stanislaus,California,70193,1112.00,6099,CA,County,550660,12747.07,201.94,201.94,12747.07,5,9,283.00,3.00
69485,2021-08-18,San Joaquin,California,85520,1489.00,6077,CA,County,762148,11220.92,195.37,195.37,11220.92,6,16,213.00,1.00
40392,2021-08-18,Riverside,California,331619,4689.00,6065,CA,County,2470546,13422.90,189.80,189.80,13422.90,7,7,1277.00,3.00
108438,2021-08-18,Tulare,California,62710,856.00,6107,CA,County,466195,13451.45,183.61,183.61,13451.45,8,6,200.00,0.00
626877,2021-08-18,Merced,California,34572,492.00,6047,CA,County,277680,12450.30,177.18,177.18,12450.30,9,10,14.00,0.00
39332,2021-08-18,Fresno,California,119941,1755.00,6019,CA,County,999101,12004.89,175.66,175.66,12004.89,10,12,640.00,2.00


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_case_df

,date,county,cases,deaths,current_crate,current_drate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...
5182,08/14/21,Lassen,6015,26.00,19674.22,85.04,35,1,0.00,0.00
5183,08/15/21,Lassen,6015,26.00,19674.22,85.04,35,1,0.00,0.00
5184,08/16/21,Lassen,6015,26.00,19674.22,85.04,35,1,0.00,0.00
5185,08/17/21,Lassen,6080,26.00,19886.83,85.04,35,1,65.00,0.00


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_death_df

,date,county,cases,deaths,current_crate,current_drate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...
5268,08/14/21,Fresno,118316,1753.00,11842.25,175.46,10,12,445.00,0.00
5269,08/15/21,Fresno,118734,1753.00,11884.08,175.46,10,12,418.00,0.00
5270,08/16/21,Fresno,119060,1753.00,11916.71,175.46,10,12,326.00,0.00
5271,08/17/21,Fresno,119301,1753.00,11940.83,175.46,10,12,241.00,0.00


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)